The file contains the adjacency list representation of a simple undirected graph. There are 200 vertices labeled 1 to 200. The first column in the file represents the vertex label, and the particular row (other entries except the first column) tells all the vertices that the vertex is adjacent to. So for example, the 6^{th}6 
th
  row looks like : "6	155	56	52	120	......". This just means that the vertex with label 6 is adjacent to (i.e., shares an edge with) the vertices with labels 155,56,52,120,......,etc

Your task is to code up and run the randomized contraction algorithm for the min cut problem and use it on the above graph to compute the min cut. (HINT: Note that you'll have to figure out an implementation of edge contractions. Initially, you might want to do this naively, creating a new graph from the old every time there's an edge contraction. But you should also think about more efficient implementations.) (WARNING: As per the video lectures, please make sure to run the algorithm many times with different random seeds, and remember the smallest cut that you ever find.) Write your numeric answer in the space provided. So e.g., if your answer is 5, just type 5 in the space provided.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

plt.style.use("ggplot")

%matplotlib inline

In [16]:
file = open("data/mincut.txt").read()

In [30]:
listed = file.split(sep = "\n")[:-1]

In [41]:
dictionary = {}
for element in listed:
    listed_element = [int(x) for x in element.split(sep="\t")[:-1]]
    dictionary[listed_element[0]] = listed_element[1:]

In [42]:
dictionary

{1: [37,
  79,
  164,
  155,
  32,
  87,
  39,
  113,
  15,
  18,
  78,
  175,
  140,
  200,
  4,
  160,
  97,
  191,
  100,
  91,
  20,
  69,
  198,
  196],
 2: [123,
  134,
  10,
  141,
  13,
  12,
  43,
  47,
  3,
  177,
  101,
  179,
  77,
  182,
  117,
  116,
  36,
  103,
  51,
  154,
  162,
  128,
  30],
 3: [48,
  123,
  134,
  109,
  41,
  17,
  159,
  49,
  136,
  16,
  130,
  141,
  29,
  176,
  2,
  190,
  66,
  153,
  157,
  70,
  114,
  65,
  173,
  104,
  194,
  54],
 4: [91,
  171,
  118,
  125,
  158,
  76,
  107,
  18,
  73,
  140,
  42,
  193,
  127,
  100,
  84,
  121,
  60,
  81,
  99,
  80,
  150,
  55,
  1,
  35,
  23,
  93],
 5: [193,
  156,
  102,
  118,
  175,
  39,
  124,
  119,
  19,
  99,
  160,
  75,
  20,
  112,
  37,
  23,
  145,
  135,
  146,
  73,
  35],
 6: [155,
  56,
  52,
  120,
  131,
  160,
  124,
  119,
  14,
  196,
  144,
  25,
  75,
  76,
  166,
  35,
  87,
  26,
  20,
  32,
  23],
 7: [156,
  185,
  178,
  79,
  27,
  52,
  144,
  107,
  78,
 